In [1]:
import os
from pysnail import Dataset, qsmooth
import pandas as pd



INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
#read the data
index_col = 0
header = 0

filename_gtex = 'panda_pancreas_GTEx.csv'
expression_gtex = pd.read_csv('expression/'+ filename_gtex,  index_col=index_col, header=header, sep=',')

filename_tcga = 'panda_pancreas_TCGA.csv'
expression_tcga = pd.read_csv('expression/'+ filename_tcga,  index_col=index_col, header=header, sep=',')

In [33]:


expression_joined = pd.concat([expression_gtex,expression_tcga],axis = 1)

group_str_gtex = ["gtex" for x in range(len(expression_gtex.columns))]
group_str_tcga = ["tcga" for x in range(len(expression_tcga.columns))]
groups_list = group_str_gtex + group_str_tcga

columns_joined = expression_gtex.columns.to_list() + expression_tcga.columns.to_list()

groups_series  = pd.Series(groups_list,index=columns_joined)


In [34]:
xprs = os.path.realpath('sample_data/expression.tsv')
groups = os.path.realpath('sample_data/groups.tsv')
dataset = Dataset(expression_joined, groups_series)

xprs_norm, qstat = qsmooth(dataset, aggregation='auto', threshold=0.2)
# xprs_norm.to_csv('pysnail_out.tsv', sep='\t')

/Users/panos/git/PySNAIL/pysnail/qsmooth.py:237: RuntimeWarning: Aggregation is set to auto, ignored argument 'threshold'.
  warnings.warn(message, RuntimeWarning)
Compute Qstat: 100%|██████████| 2/2 [00:00<00:00,  2.30it/s]
/Users/panos/git/PySNAIL/pysnail/qsmooth.py:127: RuntimeWarning: invalid value encountered in true_divide
  roughWeights = 1 - SSB / SST
Qsmooth Normalization: 100%|██████████| 2/2 [00:11<00:00,  5.80s/it]


In [69]:
gtex_norm=xprs_norm.droplevel(0, axis=1).iloc[:,:len(group_str_gtex)]
tcga_norm=xprs_norm.droplevel(0, axis=1).iloc[:,len(group_str_gtex):]



In [74]:


gtex_norm.to_csv("expression/xprs_panc_gtex_snail.tsv", sep='\t',header=True, index=True)
tcga_norm.to_csv("expression/xprs_panc_tcga_snail.tsv", sep='\t',header=True, index=True)
